<a href="https://colab.research.google.com/github/Rishitha110506/IBS-02/blob/main/IBS_2_ENDSEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LAB 1


In [ ]:
!pip install biopython

In [ ]:
!pip install biopython scikit-learn

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving alzheimers_dataset (1).fasta to alzheimers_dataset (1) (2).fasta


In [ ]:
import os
os.listdir()


['.config',
 'alzheimers_dataset (1) (1).fasta',
 'alzheimers_dataset (1).fasta',
 'alzheimers_dataset (1) (2).fasta',
 'sample_data']

In [ ]:
from Bio import SeqIO, pairwise2
import numpy as np
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity

# =========================
# 1️⃣ Load FASTA file
# =========================

fasta_file = "alzheimers_dataset (1).fasta"

sequences = []
ids = []

for record in SeqIO.parse(fasta_file, "fasta"):
    sequences.append(str(record.seq))
    ids.append(record.id)

print("Total sequences BEFORE redundancy removal:", len(sequences))

# =========================
# 2️⃣ Convert sequences to k-mer vectors
# =========================

def kmer_vector(seq, k=3):
    kmers = [seq[i:i+k] for i in range(len(seq)-k+1)]
    counts = Counter(kmers)
    return counts

all_kmers = set()
kmer_counts = []

for seq in sequences:
    counts = kmer_vector(seq)
    kmer_counts.append(counts)
    all_kmers.update(counts.keys())

all_kmers = sorted(list(all_kmers))

matrix = []

for counts in kmer_counts:
    vector = [counts.get(kmer, 0) for kmer in all_kmers]
    matrix.append(vector)

matrix = np.array(matrix)

# =========================
# 3️⃣ Cosine similarity redundancy removal
# =========================

threshold = 0.90
keep_indices = []

for i in range(len(matrix)):
    keep = True
    for j in keep_indices:
        sim = cosine_similarity([matrix[i]], [matrix[j]])[0][0]
        if sim >= threshold:
            keep = False
            break
    if keep:
        keep_indices.append(i)

unique_sequences = [sequences[i] for i in keep_indices]

print("Total sequences AFTER cosine similarity filtering:", len(unique_sequences))

# =========================
# 4️⃣ Global alignment scores (first 20)
# =========================

print("\nGlobal Alignment Scores (first 20 pairs):")

scores = []

for i in range(min(20, len(unique_sequences)-1)):
    score = pairwise2.align.globalxx(
        unique_sequences[i],
        unique_sequences[i+1],
        score_only=True
    )
    scores.append(score)
    print(f"Seq{i} vs Seq{i+1} Score:", score)

# =========================
# 5️⃣ Average alignment score
# =========================

average_score = np.mean(scores)
print("\nAverage Global Alignment Score:", average_score)

print("\nPipeline completed successfully ✅")

Total sequences BEFORE redundancy removal: 1229
Total sequences AFTER cosine similarity filtering: 496

Global Alignment Scores (first 20 pairs):
Seq0 vs Seq1 Score: 304.0
Seq1 vs Seq2 Score: 25.0
Seq2 vs Seq3 Score: 31.0
Seq3 vs Seq4 Score: 17.0
Seq4 vs Seq5 Score: 20.0
Seq5 vs Seq6 Score: 13.0
Seq6 vs Seq7 Score: 17.0
Seq7 vs Seq8 Score: 63.0
Seq8 vs Seq9 Score: 38.0
Seq9 vs Seq10 Score: 9.0
Seq10 vs Seq11 Score: 29.0
Seq11 vs Seq12 Score: 42.0
Seq12 vs Seq13 Score: 28.0
Seq13 vs Seq14 Score: 6.0
Seq14 vs Seq15 Score: 7.0
Seq15 vs Seq16 Score: 23.0
Seq16 vs Seq17 Score: 65.0
Seq17 vs Seq18 Score: 180.0
Seq18 vs Seq19 Score: 128.0
Seq19 vs Seq20 Score: 263.0

Average Global Alignment Score: 65.4

Pipeline completed successfully ✅


LAB 2

In [ ]:
# =========================
# Save non‑redundant FASTA
# =========================

output_file = "alzheimers_nonredundant.fasta"

with open(output_file, "w") as f:
    for idx, seq in enumerate(unique_sequences):
        f.write(f">NR_seq_{idx}\n")
        f.write(seq + "\n")

print("Non‑redundant FASTA saved successfully ✅")

Non‑redundant FASTA saved successfully ✅


In [ ]:
from google.colab import files
files.download("alzheimers_nonredundant.fasta")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# =========================
# AAC Feature Extraction
# =========================

amino_acids = "ACDEFGHIKLMNPQRSTVWY"

def compute_aac(seq):
    length = len(seq)
    return [seq.count(aa) / length for aa in amino_acids]

aac_features = [compute_aac(seq) for seq in unique_sequences]

print("AAC feature vector length:", len(aac_features[0]))
print("Number of sequences:", len(aac_features))

AAC feature vector length: 20
Number of sequences: 496


In [ ]:
!pip install biopython pandas numpy torch transformers openpyxl tqdm


In [ ]:
import numpy as np
import pandas as pd
import torch
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

# ================================
# SETTINGS
# ================================
fasta_file = "alzheimers_nonredundant.fasta"
output_excel = "alzheimers_features_fast.xlsx"
BATCH_SIZE = 8  # increase if GPU available

amino_acids = "ACDEFGHIKLMNPQRSTVWY"
dipeptides = [a + b for a in amino_acids for b in amino_acids]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ================================
# LOAD ProtBERT (FAST)
# ================================
print("Loading ProtBERT...")
tokenizer = AutoTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False)
model = AutoModel.from_pretrained("Rostlab/prot_bert").to(device)
model.eval()

# ================================
# AAC
# ================================
def compute_aac(seq):
    length = len(seq)
    return [seq.count(aa) / length for aa in amino_acids]

# ================================
# DIPEPTIDE
# ================================
def compute_dipeptide(seq):
    length = len(seq) - 1
    counts = {dp: 0 for dp in dipeptides}

    for i in range(length):
        dp = seq[i:i+2]
        if dp in counts:
            counts[dp] += 1

    if length > 0:
        return [counts[dp] / length for dp in dipeptides]
    else:
        return [0]*400

# ================================
# PHYSICOCHEMICAL FEATURES
# ================================
def compute_physicochemical(seq):
    try:
        analysed = ProteinAnalysis(seq)

        hydrophobicity = analysed.gravy()
        mol_weight = analysed.molecular_weight()
        pI = analysed.isoelectric_point()
        aromaticity = analysed.aromaticity()
        instability = analysed.instability_index()
        gravy = analysed.gravy()

        return [
            hydrophobicity,
            mol_weight,
            pI,
            aromaticity,
            instability,
            gravy
        ]
    except:
        return [0,0,0,0,0,0]

# ================================
# LOAD SEQUENCES
# ================================
print("Loading sequences...")

sequences = []
seq_ids = []

for record in SeqIO.parse(fasta_file, "fasta"):
    seq = str(record.seq).upper()
    sequences.append(seq)
    seq_ids.append(record.id)

print("Total sequences:", len(sequences))

# ================================
# FAST ProtBERT (BATCHED)
# ================================
def compute_bert_batch(seq_batch):

    seq_batch_spaced = [" ".join(list(seq)) for seq in seq_batch]

    inputs = tokenizer(
        seq_batch_spaced,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=1024
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
    return embeddings

# ================================
# MAIN FEATURE EXTRACTION
# ================================
all_features = []

print("Extracting features...")

for i in tqdm(range(0, len(sequences), BATCH_SIZE)):

    batch_seqs = sequences[i:i+BATCH_SIZE]

    # ---- BERT batch ----
    bert_batch = compute_bert_batch(batch_seqs)

    # ---- per-sequence features ----
    for j, seq in enumerate(batch_seqs):

        aac = compute_aac(seq)
        dipep = compute_dipeptide(seq)
        physchem = compute_physicochemical(seq)
        bert_vec = bert_batch[j]

        fused = np.concatenate([aac, dipep, physchem, bert_vec])
        all_features.append(fused)

print("✅ Feature extraction complete!")

# ================================
# COLUMN NAMES
# ================================
aac_cols = [f"AAC_{aa}" for aa in amino_acids]
dipep_cols = [f"DP_{dp}" for dp in dipeptides]
phys_cols = [
    "Hydrophobicity",
    "Molecular_Weight",
    "Isoelectric_Point",
    "Aromaticity",
    "Instability_Index",
    "GRAVY"
]
bert_dim = len(bert_vec)
bert_cols = [f"BERT_{i}" for i in range(bert_dim)]

columns = aac_cols + dipep_cols + phys_cols + bert_cols

# ================================
# SAVE EXCEL
# ================================
df = pd.DataFrame(all_features, columns=columns)
df.insert(0, "Sequence_ID", seq_ids)

df.to_excel(output_excel, index=False)

print(f"🎉 Excel saved: {output_excel}")
print("Final shape:", df.shape)

Using device: cpu
Loading ProtBERT...


Loading weights:   0%|          | 0/487 [00:00<?, ?it/s]

BertModel LOAD REPORT from: Rostlab/prot_bert
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
cls.predictions.transform.dense.weight     | UNEXPECTED |  | 
cls.seq_relationship.weight                | UNEXPECTED |  | 
cls.predictions.bias                       | UNEXPECTED |  | 
cls.predictions.transform.dense.bias       | UNEXPECTED |  | 
cls.predictions.decoder.weight             | UNEXPECTED |  | 
cls.seq_relationship.bias                  | UNEXPECTED |  | 
cls.predictions.decoder.bias               | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Loading sequences...
Total sequences: 496
Extracting features...


100%|██████████| 62/62 [1:41:27<00:00, 98.18s/it] 


✅ Feature extraction complete!
🎉 Excel saved: alzheimers_features_fast.xlsx
Final shape: (496, 1451)


In [ ]:
from google.colab import files
files.download("alzheimers_features_fast.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
